Our code to convert mesh objects to occupancy grids. We tried two different ways. First is to sample points from the meshes with the code we wrote for the exercises and then using open3d to generate voxels. The other method is to use open3d function to directly convert meshes to voxels. Finally we used the second method to generate our voxels

In [1]:
!pip install K3D>=2.9.4
!pip install matplotlib>=3.4.1
!pip install trimesh>=3.9.14
!pip install scikit-image>=0.18.1
!pip install pyrender>=0.1.43
!pip install moviepy>=1.0.3
!pip install pillow>=7.2.0
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 1.12.11 which is incompatible.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.


In [2]:
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
import os
from PIL import Image
import random
import numpy as np
import torchvision.transforms as transforms
import k3d
from matplotlib import cm, colors
import trimesh
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
objects_main_dir = '../input/modelnetdataset/ModelNet40/ModelNet40'

In [4]:
def get_vertices_faces_from_mesh(object_file_path):
    with open(object_file_path, 'r') as objfile:
        lines = [l.strip() for l in objfile.readlines()]

    start_indx = 2
    try:
        num_vertices, num_faces, _ =  [int(n) for n in lines[1].split(' ')]
    except:
        fixed_line_0 = lines[0].replace('OFF', '')
        num_vertices, num_faces, _ =  [int(n) for n in fixed_line_0.split(' ')]
        start_indx = 1

    vertices_str = lines[start_indx:num_vertices+start_indx]
    faces_str = lines[num_vertices+start_indx:]

    vertices = np.array([[float(x) for x in l.split(' ')] for l in vertices_str])
    faces = np.array([[int(x) for x in l.split(' ')] for l in faces_str])[:, 1:]
    return vertices, faces

In [5]:
voxel_grid_sizes = []
def sample_point_cloud(vertices, faces, n_points):
    """
    Sample n_points uniformly from the mesh represented by vertices and faces
    :param vertices: Nx3 numpy array of mesh vertices
    :param faces: Mx3 numpy array of mesh faces
    :param n_points: number of points to be sampled
    :return: sampled points, a numpy array of shape (n_points, 3)
    """
    # For each triangular calculate its area
    triangle_areas = []
    for face in faces:
        vertex = vertices[face]
        edge_len_1 = (np.sum((vertex[0] - vertex[1])**2)) ** 0.5
        edge_len_2 = (np.sum((vertex[0] - vertex[2])**2)) ** 0.5
        edge_len_3 = (np.sum((vertex[1] - vertex[2])**2)) ** 0.5

        # Use Heron's Formula
        s = (edge_len_1 + edge_len_2 + edge_len_3) / 2
        # When s is really small then substracting it will give 0 
        # Eventually that 0 will lead to NaN in sqrt
        s_edge_1 = max(0.0, s-edge_len_1)
        s_edge_2 = max(0.0, s-edge_len_2)
        s_edge_3 = max(0.0, s-edge_len_3)
        
        area = np.sqrt(s*s_edge_1*s_edge_2*s_edge_3)
        if np.isnan(area):
            print(edge_len_1, edge_len_2, edge_len_3)
            print(s)
            print(s-edge_len_1, s-edge_len_2, s-edge_len_3)
        triangle_areas.append(area)
    
    # Normalize areas to turn them into probability
    triangle_area_probs = triangle_areas / np.sum(triangle_areas)
    
    triangle_idxs = np.random.choice(np.arange(faces.shape[0]), n_points, p=triangle_area_probs)
    sampled_points = []
    for idx in triangle_idxs:
        A,B,C = vertices[faces[idx]]
        r1 = np.random.random()
        r2 = np.random.random()

        u = 1 - r1**0.5
        v = (r1**0.5) * (1-r2)
        w = (r1**0.5) * r2

        P = u*A + v*B + w*C
        sampled_points.append(P)
    return np.array(sampled_points)

def point_cloud_to_indices(point_cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    v_size=round(max(pcd.get_max_bound()-pcd.get_min_bound())*0.005,4)
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=v_size)
    voxels=voxel_grid.get_voxels()
    indices = np.stack(list(vx.grid_index for vx in voxels))
    return indices

In [6]:
def visualize_mesh(vertices, faces, flip_axes=False):
    plot = k3d.plot(name='points', grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    if flip_axes:
        vertices[:, 2] = vertices[:, 2] * -1
        vertices[:, [0, 1, 2]] = vertices[:, [0, 2, 1]]
    plt_mesh = k3d.mesh(vertices.astype(np.float32), faces.astype(np.uint32), color=0xd0d0d0)
    plot += plt_mesh
    plt_mesh.shader = '3d'
    plot.display()

In [7]:
def create_indices_from_mesh(faces, vertices):
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
    
    #visualize_mesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles))
    
    voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=1/32)
    voxel_indices = voxel_grid.get_voxels()
    indices = np.stack(list(vx.grid_index for vx in voxel_indices))
    return indices

In [8]:
def read_files_as_indices_from_mesh(obj_file):
    vertices, faces = get_vertices_faces_from_mesh(obj_file)
    voxel_indices = create_indices_from_mesh(faces, vertices)
    return voxel_indices

In [9]:
def read_files_as_indices(obj_file_path, N_POINTS = 5000):
    vertices, faces = get_vertices_faces_from_mesh(obj_file_path)
    int_faces = faces.astype(np.int16)
    point_cloud = sample_point_cloud(vertices, int_faces, N_POINTS)
    voxel_indices = point_cloud_to_indices(point_cloud)
    return voxel_indices

In [10]:
def voxel_indices_to_grid(indices_arr, size=34):        
    np_voxel_grid = np.zeros((size,size,size))
    for index in indices_arr:
        np_voxel_grid[index[0], index[1], index[2]] = 1
    return np_voxel_grid

In [11]:
def visualize_pointcloud(point_cloud, point_size, flip_axes=False, name='point_cloud'):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    if flip_axes:
        point_cloud[:, 2] = point_cloud[:, 2] * -1
        point_cloud[:, [0, 1, 2]] = point_cloud[:, [0, 2, 1]]
    plt_points = k3d.points(positions=point_cloud.astype(np.float32), point_size=point_size, color=0xd0d0d0)
    plot += plt_points
    plt_points.shader = '3d'
    plot.display()

In [12]:
fpath = '../input/modelnetdataset/ModelNet40/ModelNet40/bottle/test/bottle_0336.off'
indices = read_files_as_indices_from_mesh(fpath)
voxels = voxel_indices_to_grid(indices)
np_resized_voxels = np.resize(voxels, (32, 32, 32))
print(np_resized_voxels.shape)
print(np.unique(np_resized_voxels))
point_list = np.concatenate([c[:, np.newaxis] for c in np.where(np_resized_voxels)], axis=1)
visualize_pointcloud(point_list, 1)

fpath = '../input/modelnetdataset/ModelNet40/ModelNet40/night_stand/test/night_stand_0206.off'
indices = read_files_as_indices_from_mesh(fpath)
voxels = voxel_indices_to_grid(indices)
np_resized_voxels = np.resize(voxels, (32, 32, 32))
print(np_resized_voxels.shape)
point_list = np.concatenate([c[:, np.newaxis] for c in np.where(np_resized_voxels)], axis=1)
visualize_pointcloud(point_list, 1)

fpath = '../input/modelnetdataset/ModelNet40/ModelNet40/lamp/test/lamp_0139.off'
indices = read_files_as_indices_from_mesh(fpath)
voxels = voxel_indices_to_grid(indices)
point_list = np.concatenate([c[:, np.newaxis] for c in np.where(voxels)], axis=1)
visualize_pointcloud(point_list, 1)

(32, 32, 32)
[0. 1.]


Output()

(32, 32, 32)


Output()

Output()

In [13]:
dest_dir = 'model_net_voxel_indices'
os.makedirs(dest_dir, exist_ok=True)

In [14]:
for class_name in os.listdir(objects_main_dir):
    break
    print(class_name)
    for subset in ['test', 'train']:
        error_samples = 0
        obj_files_path = os.path.join(objects_main_dir, class_name, subset)
        num_objects = len(os.listdir(obj_files_path))
        
        dest_object_folder = os.path.join(dest_dir, class_name, subset)
        os.makedirs(dest_object_folder, exist_ok=True)
        
        for obj_name in os.listdir(obj_files_path):
            obj_path = os.path.join(obj_files_path, obj_name)
            f_size_in_mg = os.stat(obj_path).st_size / (1024*1024)
            if f_size_in_mg > 4:
                error_samples += 1
                continue
            try:
            # obj_indices = read_files_as_indices(obj_path)
                obj_indices = read_files_as_indices_from_mesh(obj_path) 
                dest_obj_name = obj_name.split('.')[0] + '.npy'
                object_dest_path = os.path.join(dest_object_folder, dest_obj_name)

                with open(object_dest_path, 'wb') as fb:
                    np.save(fb, obj_indices)
            except:
                error_samples += 1
        print(f"Found : {error_samples}/{num_objects} on {class_name}-{subset}")

In [15]:
!zip -r 'model_net_voxel_indices.zip' 'model_net_voxel_indices'
!rm -r 'model_net_voxel_indices'

  adding: model_net_voxel_indices/ (stored 0%)


In [16]:
os.remove('./=3.9.14')
os.remove('./=7.2.0')
os.remove('./=0.18.1')
os.remove('./=1.0.3')
os.remove('./=3.4.1')
os.remove('./=2.9.4')
os.remove('./=0.1.43')